# Relu

In [20]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
tf.enable_eager_execution()
import os

In [2]:
import tensorflow as tf
print(tf.__version__)

1.15.0-dev20190821


In [4]:
def load_mnist():
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis = -1) #add channel
    test_data = np.expand_dims(test_data, axis = -1)
    
    train_data, test_data = normalize(train_data, test_data)
    
    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)
    return train_data, train_labels, test_data, test_labels
    
def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32)/255.0
    test_data = test_data.astype(np.float32)/255.0
    return train_data, test_data
    

In [27]:
def loss_fn(model, images, labels):
    logits = model(images, training = True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels = labels))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training = False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [13]:
def flatten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init):
    return tf.keras.layers.Dense(units = label_dim, use_bias=True, kernel_initializer=weight_init)

def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

## create model

In [14]:
class create_model(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()
        
        self.model = tf.keras.Sequential()
        self.model.add(flatten()) #why? fully connected layer사용할거라
        
        for i in range(2):
            self.model.add(dense(256, weight_init)) #[N,256]
            self.model.add(relu())
            
        self.model.add(dense(label_dim, weight_init)) #[N,10]
        
    def call(self, x, training = None, mask = None):
        x = self.model(x)
        
        return x
        

In [11]:
train_x, train_y, test_x, test_y = load_mnist()

learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x)//batch_size

label_dim = 10

train_flag = True

train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
                    shuffle(buffer_size=100000).\
                    prefetch(buffer_size=batch_size).\
                    batch(batch_size).\
                    repeat()

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
                    shuffle(buffer_size=100000).\
                    prefetch(buffer_size=len(test_x)).\
                    batch(len(test_x)).\
                    repeat()

train_iterator = train_dataset.make_one_shot_iterator()
test_iterator = test_dataset.make_one_shot_iterator()

W0922 22:47:32.629638 4455216576 deprecation.py:323] From <ipython-input-11-acfae959e7eb>:25: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [15]:
network = create_model(label_dim)

In [16]:
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)

In [29]:
# prevent networks from losing connects or trained info
checkpoint = tf.train.Checkpoint(dnn=network)

for epoch in range(training_epochs):
    for idx in range(training_iterations):
        train_input, train_label = train_iterator.get_next()
        
        grads = grad(network, train_input, train_label)
        optimizer.apply_gradients(grads_and_vars = zip(grads, network.variables))
        
        train_loss = loss_fn(network, train_input, train_label)
        train_accuracy = accuracy_fn(network, train_input, train_label)
        
        test_input, test_label = test_iterator.get_next()
        test_accuracy = accuracy_fn(network, test_input, test_label)
        
        print("Epoch: [%2d] [%5d/%5d], train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f"\
             % (epoch, idx, training_iterations, train_loss, train_accuracy, test_accuracy))


Epoch: [ 0] [    0/  468], train_loss: 0.06956463, train_accuracy: 0.9766, test_Accuracy: 0.9603
Epoch: [ 0] [    1/  468], train_loss: 0.08219013, train_accuracy: 0.9766, test_Accuracy: 0.9598
Epoch: [ 0] [    2/  468], train_loss: 0.16748595, train_accuracy: 0.9609, test_Accuracy: 0.9589
Epoch: [ 0] [    3/  468], train_loss: 0.06257720, train_accuracy: 0.9844, test_Accuracy: 0.9581
Epoch: [ 0] [    4/  468], train_loss: 0.07798314, train_accuracy: 0.9688, test_Accuracy: 0.9580
Epoch: [ 0] [    5/  468], train_loss: 0.12681103, train_accuracy: 0.9531, test_Accuracy: 0.9586
Epoch: [ 0] [    6/  468], train_loss: 0.12817638, train_accuracy: 0.9453, test_Accuracy: 0.9591
Epoch: [ 0] [    7/  468], train_loss: 0.08528638, train_accuracy: 0.9609, test_Accuracy: 0.9604
Epoch: [ 0] [    8/  468], train_loss: 0.10367087, train_accuracy: 0.9766, test_Accuracy: 0.9614
Epoch: [ 0] [    9/  468], train_loss: 0.09343378, train_accuracy: 0.9766, test_Accuracy: 0.9618
Epoch: [ 0] [   10/  468], tra

Epoch: [ 0] [   86/  468], train_loss: 0.09534584, train_accuracy: 0.9766, test_Accuracy: 0.9657
Epoch: [ 0] [   87/  468], train_loss: 0.08026856, train_accuracy: 0.9844, test_Accuracy: 0.9648
Epoch: [ 0] [   88/  468], train_loss: 0.13498273, train_accuracy: 0.9688, test_Accuracy: 0.9642
Epoch: [ 0] [   89/  468], train_loss: 0.11717654, train_accuracy: 0.9688, test_Accuracy: 0.9641
Epoch: [ 0] [   90/  468], train_loss: 0.13940179, train_accuracy: 0.9609, test_Accuracy: 0.9638
Epoch: [ 0] [   91/  468], train_loss: 0.25679821, train_accuracy: 0.9219, test_Accuracy: 0.9647
Epoch: [ 0] [   92/  468], train_loss: 0.06112855, train_accuracy: 0.9922, test_Accuracy: 0.9651
Epoch: [ 0] [   93/  468], train_loss: 0.06119572, train_accuracy: 1.0000, test_Accuracy: 0.9661
Epoch: [ 0] [   94/  468], train_loss: 0.04122718, train_accuracy: 1.0000, test_Accuracy: 0.9664
Epoch: [ 0] [   95/  468], train_loss: 0.08435372, train_accuracy: 0.9609, test_Accuracy: 0.9671
Epoch: [ 0] [   96/  468], tra

Epoch: [ 0] [  172/  468], train_loss: 0.07626343, train_accuracy: 0.9766, test_Accuracy: 0.9650
Epoch: [ 0] [  173/  468], train_loss: 0.10000974, train_accuracy: 0.9688, test_Accuracy: 0.9658
Epoch: [ 0] [  174/  468], train_loss: 0.08728817, train_accuracy: 0.9766, test_Accuracy: 0.9663
Epoch: [ 0] [  175/  468], train_loss: 0.06358220, train_accuracy: 0.9844, test_Accuracy: 0.9662
Epoch: [ 0] [  176/  468], train_loss: 0.14597619, train_accuracy: 0.9531, test_Accuracy: 0.9664
Epoch: [ 0] [  177/  468], train_loss: 0.07181138, train_accuracy: 0.9844, test_Accuracy: 0.9663
Epoch: [ 0] [  178/  468], train_loss: 0.14658667, train_accuracy: 0.9609, test_Accuracy: 0.9661
Epoch: [ 0] [  179/  468], train_loss: 0.08782369, train_accuracy: 0.9844, test_Accuracy: 0.9656
Epoch: [ 0] [  180/  468], train_loss: 0.09666011, train_accuracy: 0.9531, test_Accuracy: 0.9645
Epoch: [ 0] [  181/  468], train_loss: 0.13078962, train_accuracy: 0.9609, test_Accuracy: 0.9635
Epoch: [ 0] [  182/  468], tra

Epoch: [ 0] [  258/  468], train_loss: 0.04478443, train_accuracy: 0.9922, test_Accuracy: 0.9665
Epoch: [ 0] [  259/  468], train_loss: 0.05370596, train_accuracy: 0.9766, test_Accuracy: 0.9669
Epoch: [ 0] [  260/  468], train_loss: 0.08997103, train_accuracy: 0.9844, test_Accuracy: 0.9671
Epoch: [ 0] [  261/  468], train_loss: 0.05800814, train_accuracy: 0.9844, test_Accuracy: 0.9663
Epoch: [ 0] [  262/  468], train_loss: 0.11432949, train_accuracy: 0.9844, test_Accuracy: 0.9669
Epoch: [ 0] [  263/  468], train_loss: 0.10898130, train_accuracy: 0.9609, test_Accuracy: 0.9679
Epoch: [ 0] [  264/  468], train_loss: 0.05299885, train_accuracy: 0.9844, test_Accuracy: 0.9676
Epoch: [ 0] [  265/  468], train_loss: 0.14956161, train_accuracy: 0.9453, test_Accuracy: 0.9683
Epoch: [ 0] [  266/  468], train_loss: 0.08575435, train_accuracy: 0.9688, test_Accuracy: 0.9684
Epoch: [ 0] [  267/  468], train_loss: 0.05602010, train_accuracy: 0.9844, test_Accuracy: 0.9683
Epoch: [ 0] [  268/  468], tra

Epoch: [ 0] [  344/  468], train_loss: 0.10336490, train_accuracy: 0.9531, test_Accuracy: 0.9689
Epoch: [ 0] [  345/  468], train_loss: 0.13788712, train_accuracy: 0.9766, test_Accuracy: 0.9684
Epoch: [ 0] [  346/  468], train_loss: 0.09423746, train_accuracy: 0.9844, test_Accuracy: 0.9691
Epoch: [ 0] [  347/  468], train_loss: 0.07222025, train_accuracy: 0.9766, test_Accuracy: 0.9695
Epoch: [ 0] [  348/  468], train_loss: 0.04806850, train_accuracy: 0.9922, test_Accuracy: 0.9701
Epoch: [ 0] [  349/  468], train_loss: 0.12105203, train_accuracy: 0.9688, test_Accuracy: 0.9704
Epoch: [ 0] [  350/  468], train_loss: 0.07495613, train_accuracy: 0.9766, test_Accuracy: 0.9709
Epoch: [ 0] [  351/  468], train_loss: 0.05205756, train_accuracy: 0.9844, test_Accuracy: 0.9704
Epoch: [ 0] [  352/  468], train_loss: 0.04281736, train_accuracy: 0.9922, test_Accuracy: 0.9696
Epoch: [ 0] [  353/  468], train_loss: 0.09503687, train_accuracy: 0.9609, test_Accuracy: 0.9684
Epoch: [ 0] [  354/  468], tra

Epoch: [ 0] [  430/  468], train_loss: 0.07842873, train_accuracy: 0.9766, test_Accuracy: 0.9670
Epoch: [ 0] [  431/  468], train_loss: 0.09824622, train_accuracy: 0.9688, test_Accuracy: 0.9669
Epoch: [ 0] [  432/  468], train_loss: 0.07792479, train_accuracy: 0.9922, test_Accuracy: 0.9681
Epoch: [ 0] [  433/  468], train_loss: 0.07401152, train_accuracy: 0.9766, test_Accuracy: 0.9674
Epoch: [ 0] [  434/  468], train_loss: 0.11601779, train_accuracy: 0.9844, test_Accuracy: 0.9693
Epoch: [ 0] [  435/  468], train_loss: 0.06881586, train_accuracy: 0.9922, test_Accuracy: 0.9703
Epoch: [ 0] [  436/  468], train_loss: 0.03121004, train_accuracy: 0.9922, test_Accuracy: 0.9707
Epoch: [ 0] [  437/  468], train_loss: 0.04870085, train_accuracy: 1.0000, test_Accuracy: 0.9713
Epoch: [ 0] [  438/  468], train_loss: 0.06146710, train_accuracy: 0.9844, test_Accuracy: 0.9723
Epoch: [ 0] [  439/  468], train_loss: 0.08991017, train_accuracy: 0.9766, test_Accuracy: 0.9730
Epoch: [ 0] [  440/  468], tra

In [30]:
print("test_Accuracy: %.4f" % (test_accuracy))

test_Accuracy: 0.9736


In [ ]:
checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))